In [1]:
import math
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.distributions
from collections import namedtuple
from itertools import count

device = "cuda:0"
floattype = torch.float

batchsize = 512
nsamples = 8
npoints = 5
emsize = 512


class Graph_Transformer(nn.Module):
    def __init__(self, emsize = 64, nhead = 8, nhid = 1024, nlayers = 3, ndecoderlayers = 0, dropout = 0):
        super().__init__()
        self.emsize = emsize
        from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer
        encoder_layers = TransformerEncoderLayer(emsize, nhead, nhid, dropout = dropout)
        decoder_layers = TransformerDecoderLayer(emsize, nhead, nhid, dropout = dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.transformer_decoder = TransformerDecoder(decoder_layers, ndecoderlayers)
        self.encoder = nn.Linear(3, emsize)
        self.outputattention_query = nn.Linear(emsize, emsize, bias = False)
        self.outputattention_key = nn.Linear(emsize, emsize, bias = False)
        self.start_token = nn.Parameter(torch.randn([emsize], device = device))
    
    def generate_subsequent_mask(self, sz): #last dimension will be softmaxed over when adding to attention logits, if boolean the ones turn into -inf
        #mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        #mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        
        mask = torch.triu(torch.ones([sz, sz], dtype = torch.bool, device = device), diagonal = 1)
        return mask
    
    def encode(self, src): #src must be [batchsize * nsamples, npoints, 3]
        src = self.encoder(src).transpose(0, 1)
        output = self.transformer_encoder(src)
        return output #[npoints, batchsize * nsamples, emsize]
    
    def decode_next(self, memory, tgt, route_mask): #route mask is [batchsize * nsamples, npoints], both memory and tgt must have batchsize and nsamples in same dimension (the 1th one)
        npoints = memory.size(0)
        batchsize = tgt.size(1)
        """if I really wanted this to be efficient I'd only recompute the decoder for the last tgt, and just remebering what the others looked like from before (won't change due to mask)"""
        """have the option to freeze the autograd on all but the last part of tgt, although at the moment this is a very natural way to say: initial choices matter more"""
        tgt_mask = self.generate_subsequent_mask(tgt.size(0))
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[tgt, batchsize * nsamples, emsize]
        output_query = self.outputattention_query(memory).transpose(0, 1) #[batchsize * nsamples, npoints, emsize]
        output_key = self.outputattention_key(output[-1]) #[batchsize * nsamples, emsize]
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.unsqueeze(-1)).squeeze(-1) #[batchsize * nsamples, npoints], technically don't need to scale attention as we divide by variance next anyway
        output_attention_tanh = output_attention.tanh() #[batchsize * nsamples, npoints]
        
        #we clone the route_mask incase we want to backprop using it (else it was modified by inplace opporations)
        output_attention = output_attention.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        output_attention_tanh = output_attention_tanh.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        
        return output_attention_tanh, output_attention #[batchsize * nsamples, npoints]
    
    def calculate_logprob(self, memory, routes): #memory is [npoints, batchsize * nsamples, emsize], routes is [batchsize * nsamples, npoints - 4], rather than backproping the entire loop, this saves vram (and computation)
        npoints = memory.size(0)
        ninternalpoints = routes.size(1)
        bigbatchsize = memory.size(1)
        memory_ = memory.gather(0, routes.transpose(0, 1).unsqueeze(2).expand(-1, -1, self.emsize)) #[npoints - 4, batchsize * nsamples, emsize] reorder memory into order of routes
        tgt = torch.cat([self.start_token.unsqueeze(0).unsqueeze(1).expand(1, bigbatchsize, -1), memory_[:-1]]) #[npoints - 4, batchsize * nroutes, emsize], want to go from memory to tgt
        tgt_mask = self.generate_subsequent_mask(ninternalpoints)
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[npoints - 4, batchsize * nsamples, emsize]
        """want probability of going from key to query, but first need to normalise (softmax with mask)"""
        output_query = self.outputattention_query(memory_).transpose(0, 1) #[batchsize * nsamples, npoints - 4, emsize]
        output_key = self.outputattention_key(output).transpose(0, 1) #[batchsize * nsamples, npoints - 4, emsize]
        attention_mask = torch.full([ninternalpoints, ninternalpoints], True, device = device).triu(1) #[npoints - 4, npoints - 4], True for i < j
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.transpose(-1, -2))
        """quick fix to stop divergence"""
        output_attention_tanh = output_attention.tanh()
        
        output_attention_tanh = output_attention_tanh.masked_fill(attention_mask, float('-inf'))
        output_attention_tanh = output_attention_tanh - output_attention_tanh.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 4, npoints - 4]
        
        output_attention = output_attention.masked_fill(attention_mask, float('-inf'))
        output_attention = output_attention - output_attention.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 4, npoints - 4]
        
        """infact I'm almost tempted to not mask choosing a previous point, so it's forced to learn it and somehow incorporate it into its computation, but without much impact on reinforcing good examples"""
        logprob_tanh = output_attention_tanh.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        logprob = output_attention.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        return logprob_tanh, logprob #[batchsize * nsamples]

NN = Graph_Transformer().to(device)
optimizer = optim.Adam(NN.parameters())


class environment:    
    def reset(self, npoints, batchsize, nsamples=1, corner_points = None, initial_triangulation = None):
        """
        corner_points, etc., shoudn't include a batch dimension
        """
        if corner_points == None:
            ncornerpoints = 4
        else:
            ncornerpoints = corner_points.size(0)
        if npoints <= ncornerpoints:
            print("Error: not enough points for valid problem instance")
            return
        self.batchsize = (
            batchsize * nsamples
        )  # so that I don't have to rewrite all this code, we store these two dimensions together
        self.nsamples = nsamples
        self.npoints = npoints
        self.points = (
            torch.rand([batchsize, npoints - ncornerpoints, 3], dtype = floattype, device=device)
            .unsqueeze(1)
            .expand(-1, nsamples, -1, -1)
            .reshape(self.batchsize, npoints - ncornerpoints, 3)
        )
        if corner_points == None:
            self.corner_points = torch.tensor(
                [[0, 0, 0], [3, 0, 0], [0, 3, 0], [0, 0, 3]], dtype = floattype, device=device
            )
        else:
            self.corner_points = corner_points
        self.points = torch.cat(
            [
                self.corner_points.unsqueeze(0).expand(self.batchsize, -1, -1),
                self.points,
            ],
            dim=-2,
        )  # [batchsize * nsamples, npoints, 3]
        self.points_mask = torch.cat(
            [
                torch.ones([self.batchsize, ncornerpoints], dtype=torch.bool, device=device),
                torch.zeros(
                    [self.batchsize, npoints - ncornerpoints], dtype=torch.bool, device=device
                ),
            ],
            dim=1,
        )
        self.points_sequence = torch.empty(
            [self.batchsize, 0], dtype=torch.long, device=device
        )

        """
        points are now triples
        triangles are now quadruples
        edges are now still just indices, but there are four of them per 'triangle', and they correspond to triples of points, not pairs
        we use  0,2,1  0,3,2  0,1,3  1,2,3  as the order of the four 'edges'/faces
        opposite face is always ordered such that the last two indices are swapped
        faces are always read ANTICLOCKWISE
        
        first three points of tetrahedron MUST be read clockwise (from the outside) to get correct sign on incircle test
        
        new point will be inserted in zeroth position, so if corresponding face of REMOVED tetrahedron is [x,y,z] (being read anticlockwise from outside in) new tetrahedron is [p, x, y, z]
        """
        
        """
        number of tetrahedra is not the same for each batch (in 3D), so store as a big list, and remember batch index that it comes from
        """
        if corner_points == None:
            initial_triangulation = torch.tensor([[0, 1, 2, 3]], dtype=torch.long, device=device)
        
        self.partial_delaunay_triangles = initial_triangulation.unsqueeze(0).expand(self.batchsize, -1, -1).reshape(-1, 4)
        self.batch_index = torch.arange(self.batchsize, dtype = torch.long, device = device).unsqueeze(1).expand(-1, initial_triangulation.size(0)).reshape(-1)
        
        self.batch_triangles = self.partial_delaunay_triangles.size(0) #[0]
        self.ntriangles = torch.full([self.batchsize], initial_triangulation.size(0), dtype = torch.long, device = device) #[self.batchsize]
        
        self.cost = torch.zeros([self.batchsize], dtype = floattype, device=device)

        self.logprob = torch.zeros([self.batchsize], dtype = floattype, device=device, requires_grad=True)

    def update(self, point_index):  # point_index is [batchsize]
        
        assert point_index.size(0) == self.batchsize
        assert str(point_index.device) == device
        assert self.points_mask.gather(1, point_index.unsqueeze(1)).sum() == 0
        
        triangles_coordinates = self.points[self.batch_index.unsqueeze(1), self.partial_delaunay_triangles] # [batch_triangles, 4, 3]
        
        newpoint = self.points[self.batch_index, point_index[self.batch_index]] # [batch_triangles, 3]

        incircle_matrix = torch.cat(
            [
                newpoint.unsqueeze(1),
                triangles_coordinates,
            ],
            dim=-2,
        )  # [batch_triangles, 5, 3]
        incircle_matrix = torch.cat(
            [
                (incircle_matrix * incircle_matrix).sum(-1, keepdim=True),
                incircle_matrix,
                torch.ones([self.batch_triangles, 5, 1], dtype = floattype, device=device),
            ],
            dim=-1,
        )  # [batch_triangles, 5, 5]
        assert incircle_matrix.dtype == floattype
        assert str(incircle_matrix.device) == device
        
        incircle_test = (
            incircle_matrix.det() > 0
        )  # [batch_triangles], is True if inside incircle
        
        conflicts = incircle_test.sum()
        
        conflicting_triangles = self.partial_delaunay_triangles[incircle_test] # [conflicts, 4]
        
        conflicting_edges_index0 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([0, 0, 0, 1])
        conflicting_edges_index0 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges_index1 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([2, 3, 1, 2])
        conflicting_edges_index1 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges_index2 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([1, 2, 3, 3])
        conflicting_edges_index2 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges = torch.cat([conflicting_edges_index0.view(-1).unsqueeze(-1), conflicting_edges_index1.view(-1).unsqueeze(-1), conflicting_edges_index2.view(-1).unsqueeze(-1)], dim = -1).reshape(-1, 3) # [conflicts * 4, 3]
        
        edge_batch_index = self.batch_index[incircle_test].unsqueeze(1).expand(-1, 4).reshape(-1) # [conflicts * 4]
        
        indices = torch.LongTensor([0, 2, 1])
        comparison_edges = conflicting_edges[:, indices] # [conflicts * 4, 3]        
        
        unravel_nomatch_mask = torch.ones([conflicts * 4], dtype = torch.bool, device = device) # [conflicts * 4]
        i = 1
        while True:
            
            todo_mask = unravel_nomatch_mask[:-i].logical_and(edge_batch_index[:-i] == edge_batch_index[i:])
            if i % 4 == 0:
                if todo_mask.sum() == 0:
                    break
            
            match_mask = todo_mask.clone()
            match_mask[todo_mask] = (conflicting_edges[:-i][todo_mask] != comparison_edges[i:][todo_mask]).sum(-1).logical_not()
            
            unravel_nomatch_mask[:-i][match_mask] = False
            unravel_nomatch_mask[i:][match_mask] = False
            
            i += 1
        
        batch_newtriangles = unravel_nomatch_mask.sum()
        
        nomatch_edges = conflicting_edges[unravel_nomatch_mask] # [batch_newtriangles, 3], already in correct order to insert into 1,2,3 (since already anticlockwise from outside in)
        assert list(nomatch_edges.size()) == [batch_newtriangles, 3]
        nomatch_batch_index = edge_batch_index[unravel_nomatch_mask] # [batch_newtriangles]
        
        nomatch_newpoint = point_index[nomatch_batch_index] # [batch_newtriangles]
        
        newtriangles = torch.cat([nomatch_newpoint.unsqueeze(1), nomatch_edges], dim = -1) # [batch_newtriangles, 4]
        
        
        nremoved_triangles = torch.zeros([self.batchsize], dtype = torch.long, device = device)
        nnew_triangles = torch.zeros([self.batchsize], dtype = torch.long, device = device)
        
        indices = self.batch_index[incircle_test]
        nremoved_triangles.put_(indices, torch.ones_like(indices, dtype = torch.long), accumulate = True) # [batchsize]
        
        indices = edge_batch_index[unravel_nomatch_mask]
        nnew_triangles.put_(indices, torch.ones_like(indices, dtype = torch.long), accumulate = True) # [batchsize]
        
        assert (nnew_triangles <= 2 * nremoved_triangles + 2).logical_not().sum().logical_not()
        
        """
        NOTE:
        I THINK it's possible for nnew_triangles to be less than nremoved_triangles (or my code is just buggy...)
        """
        
        assert nnew_triangles.sum() == batch_newtriangles
        assert nremoved_triangles.sum() == incircle_test.sum()
        
        nadditional_triangles = nnew_triangles - nremoved_triangles # [batchsize]
        ntriangles = self.ntriangles + nadditional_triangles # [batchsize]
        
        partial_delaunay_triangles = torch.empty([ntriangles.sum(), 4], dtype = torch.long, device = device)
        batch_index = torch.empty([ntriangles.sum()], dtype = torch.long, device = device)
        
        cumulative_triangles = torch.cat([torch.zeros([1], dtype = torch.long, device = device), nnew_triangles.cumsum(0)[:-1]]) # [batchsize], cumulative sum starts at zero
        
        """
        since may actually have LESS triangles than previous round, we insert all that survive into the first slots (in that batch)
        """
        good_triangle_indices = torch.arange(incircle_test.logical_not().sum(), dtype = torch.long, device = device)
        good_triangle_indices += cumulative_triangles[self.batch_index[incircle_test.logical_not()]]
        bad_triangle_indices_mask = torch.ones([ntriangles.sum(0)], dtype = torch.bool, device = device)
        bad_triangle_indices_mask.scatter_(0, good_triangle_indices, False)
        
        assert good_triangle_indices.size(0) == incircle_test.logical_not().sum()
        assert bad_triangle_indices_mask.sum() == batch_newtriangles
        
        partial_delaunay_triangles[good_triangle_indices] = self.partial_delaunay_triangles[~incircle_test]
        batch_index[good_triangle_indices] = self.batch_index[~incircle_test]
        
        partial_delaunay_triangles[bad_triangle_indices_mask] = newtriangles
        batch_index[bad_triangle_indices_mask] = nomatch_batch_index
        
        self.partial_delaunay_triangles = partial_delaunay_triangles
        self.batch_index = batch_index
        
        self.ntriangles = ntriangles
        self.batch_triangles = self.partial_delaunay_triangles.size(0)
        
        self.points_mask.scatter_(
            1, point_index.unsqueeze(1).expand(-1, self.npoints), True
        )
        self.points_sequence = torch.cat(
            [self.points_sequence, point_index.unsqueeze(1)], dim=1
        )
        
        self.cost += nremoved_triangles
        return
    
    def sample_point(self, logits): #logits must be [batchsize * nsamples, npoints]
        probs = torch.distributions.categorical.Categorical(logits = logits)
        next_point = probs.sample() #size is [batchsize * nsamples]
        self.update(next_point)
        self.logprob = self.logprob + probs.log_prob(next_point)
        return next_point #[batchsize * nsamples]
    
    def sampleandgreedy_point(self, logits): #logits must be [batchsize * nsamples, npoints], last sample will be the greedy choice (but we still need to keep track of its logits)
        logits_sample = logits.view(-1, self.nsamples, self.npoints)[:, :-1, :]
        probs = torch.distributions.categorical.Categorical(logits = logits_sample)
        
        sample_point = probs.sample() #[batchsize, (nsamples - 1)]
        greedy_point = logits.view(-1, self.nsamples, self.npoints)[:, -1, :].max(-1, keepdim = True)[1] #[batchsize, 1]
        next_point = torch.cat([sample_point, greedy_point], dim = 1).view(-1)
        self.update(next_point)
        self.logprob = self.logprob + torch.cat([probs.log_prob(sample_point), torch.zeros([sample_point.size(0), 1], device = device)], dim = 1).view(-1)
        return next_point
    

env = environment()


def train(epochs = 30000, npoints = 14, batchsize = 100, nsamples = 8):
    NN.train()
    for i in range(epochs):
        env.reset(npoints, batchsize, nsamples)
        """include the boundary points, kinda makes sense that they should contribute (atm only in the encoder, difficult to see how in the decoder)"""
        memory = NN.encode(env.points) #[npoints, batchsize * nsamples, emsize]
        #### #### #### remember to include tgt.detach() when reinstate with torch.no_grad()
        tgt = NN.start_token.unsqueeze(0).unsqueeze(1).expand(1, batchsize * nsamples, -1).detach() #[1, batchsize * nsamples, emsize]
        #with torch.no_grad(): #to speed up computation, selecting routes is done without gradient
        with torch.no_grad():
            for j in range(4, npoints):
                #### #### #### remember to include memory.detach() when reinstate with torch.no_grad()
                _, logits = NN.decode_next(memory.detach(), tgt, env.points_mask)
                next_point = env.sampleandgreedy_point(logits)
                """
                for inputing the previous embedding into decoder
                """
                tgt = torch.cat([tgt, memory.gather(0, next_point.unsqueeze(0).unsqueeze(2).expand(1, -1, memory.size(2)))]) #[nsofar, batchsize * nsamples, emsize]
                """
                for inputing the previous decoder output into the decoder (allows for an evolving strategy, but doesn't allow for fast training
                """
                ############

        
        NN.eval()
        _, logprob = NN.calculate_logprob(memory, env.points_sequence) #[batchsize * nsamples]
        NN.train()
        """
        clip logprob so doesn't reinforce things it already knows
        TBH WANT SOMETHING DIFFERENT ... want to massively increase training if find something unexpected and otherwise not
        """
        greedy_prob = logprob.view(batchsize, nsamples)[:, -1].detach() #[batchsize]
        greedy_baseline = env.cost.view(batchsize, nsamples)[:, -1] #[batchsize], greedy sample
        fixed_baseline = 0.5 * torch.ones([1], device = device)
        min_baseline = env.cost.view(batchsize, nsamples)[:, :-1].min(-1)[0] #[batchsize], minimum cost
        baseline = greedy_baseline
        positive_reinforcement = - F.relu( - (env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))) #don't scale positive reinforcement
        negative_reinforcement = F.relu(env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))
        positive_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) <= -0.05
        negative_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) > 1
        """
        binary positive reinforcement
        """
        #loss = - ((logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement_binary).mean() #+ (logprob.view(batchsize, nsamples)[:, :-1] > -1) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement_binary
        """
        clipped binary reinforcement
        """
        loss = ( 
                - logprob.view(batchsize, nsamples)[:, :-1] 
                #* (logprob.view(batchsize, nsamples)[:, :-1] < 0) 
                * positive_reinforcement_binary 
                + logprob.view(batchsize, nsamples)[:, :-1] 
                #* (logprob.view(batchsize, nsamples)[:, :-1] > greedy_prob.unsqueeze(1) - 13) 
                * negative_reinforcement_binary 
        ).mean()
        """
        clipped binary postive, clipped weighted negative
        """
        #loss = ( - logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * positive_reinforcement_binary + logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] > -2) * negative_reinforcement ).mean()
        """
        clipped reinforcement without rescaling
        """
        #loss = ((logprob.view(batchsize, nsamples)[:, :-1] < -0.7) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement + (logprob.view(batchsize, nsamples)[:, :-1] > -5) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement).mean()
        """
        clipped reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + (logprob.view(batchsize, nsamples)[:, :-1] > -3) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt()).mean()
        """
        balanced reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt())).mean()
        """
        regular loss
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement + negative_reinforcement)).mean()
        optimizer.zero_grad()
        loss.backward()
        #print(NN.encoder.weight.grad)
        optimizer.step()
        #print(greedy_baseline.mean().item())
        print(greedy_baseline.mean().item(), logprob.view(batchsize, nsamples)[:, -1].mean().item(), logprob.view(batchsize, nsamples)[:, :-1].mean().item(), logprob[batchsize - 1].item(), logprob[0].item(), env.logprob[0].item())
        
        

In [2]:
train(epochs = 300000, npoints = 14, batchsize = 1750, nsamples = 8)

50.40399932861328 -14.798803329467773 -15.099384307861328 -15.299233436584473 -14.94122314453125 -14.941224098205566
45.72571563720703 -12.816160202026367 -14.690592765808105 -14.780645370483398 -16.243606567382812 -16.243606567382812
44.682857513427734 -9.544747352600098 -12.711823463439941 -12.806971549987793 -13.753508567810059 -13.753507614135742
44.80742645263672 -7.842775821685791 -10.847058296203613 -10.68731689453125 -11.816468238830566 -11.816466331481934
44.558284759521484 -5.089664936065674 -7.868050575256348 -7.89332389831543 -8.033981323242188 -8.033980369567871
44.223426818847656 -4.918994426727295 -7.52626371383667 -4.612244129180908 -4.608835697174072 -4.6088361740112305
44.503997802734375 -4.036983966827393 -6.53372859954834 -8.808395385742188 -5.597065448760986 -5.5970659255981445
44.19771194458008 -3.8866159915924072 -6.352922439575195 -4.386517524719238 -5.896528244018555 -5.89652681350708
44.36857223510742 -3.848435878753662 -6.376160621643066 -5.102601051330566 -6

42.992000579833984 -2.513500213623047 -4.303894996643066 -1.4789884090423584 -1.4992575645446777 -1.499257206916809
42.49542999267578 -2.7673983573913574 -4.651178359985352 -3.3326940536499023 -5.592053413391113 -5.592052936553955
42.68742752075195 -2.9599311351776123 -4.895810604095459 -4.505048751831055 -6.692490577697754 -6.692490577697754
42.5697135925293 -2.984157085418701 -4.930187225341797 -6.1118364334106445 -2.2622156143188477 -2.2622170448303223
42.84742736816406 -2.985286235809326 -4.919341087341309 -3.1728971004486084 -8.71381950378418 -8.71381950378418
42.55657196044922 -3.217693567276001 -5.230094909667969 -5.470928192138672 -1.8079768419265747 -1.8079776763916016
42.42399978637695 -3.489373207092285 -5.6001386642456055 -3.475508451461792 -3.971364974975586 -3.971364736557007
42.6971435546875 -3.3140780925750732 -5.394182205200195 -3.8631019592285156 -6.573370933532715 -6.573371887207031
42.32914352416992 -2.8332948684692383 -4.708094120025635 -5.368454933166504 -5.742078

40.93828582763672 -3.054931402206421 -5.126688480377197 -5.005478858947754 -6.087233543395996 -6.087233543395996
40.64285659790039 -2.966066837310791 -5.008881092071533 -5.489758014678955 -3.5987441539764404 -3.5987448692321777
40.514286041259766 -2.865504503250122 -4.861064434051514 -5.606886386871338 -4.4235076904296875 -4.423506736755371
40.50685501098633 -2.591306686401367 -4.447163105010986 -7.464441299438477 -2.0536575317382812 -2.0536582469940186
40.62114334106445 -2.4793477058410645 -4.28847074508667 -8.294964790344238 -1.697425127029419 -1.6974252462387085
40.3782844543457 -2.7459397315979004 -4.656472682952881 -3.314783811569214 -1.9133330583572388 -1.9133325815200806
40.458858489990234 -3.0789570808410645 -5.216700077056885 -5.316850662231445 -3.3791933059692383 -3.3791937828063965
40.6022834777832 -3.0922136306762695 -5.223877429962158 -9.294054985046387 -4.0521979331970215 -4.052196979522705
40.468570709228516 -2.763612747192383 -4.747493267059326 -5.679714679718018 -7.942

39.489715576171875 -2.6806466579437256 -4.674944877624512 -4.6215949058532715 -3.9908273220062256 -3.9908275604248047
39.54800033569336 -2.493192434310913 -4.375158786773682 -4.213616847991943 -2.4803085327148438 -2.4803085327148438
39.3062858581543 -2.5290541648864746 -4.4391045570373535 -7.4503583908081055 -3.8415911197662354 -3.8415911197662354
39.36800003051758 -2.4516477584838867 -4.3186259269714355 -6.024619102478027 -3.1730902194976807 -3.1730895042419434
39.59600067138672 -2.427371025085449 -4.291679859161377 -4.066415309906006 -3.2180984020233154 -3.2181005477905273
39.510284423828125 -2.5190470218658447 -4.436797142028809 -6.061176300048828 -7.536677360534668 -7.536677360534668
39.394287109375 -2.642439603805542 -4.579509258270264 -4.186453819274902 -5.379672050476074 -5.379672050476074
39.60057067871094 -2.6339805126190186 -4.6013054847717285 -1.9476282596588135 -3.4860408306121826 -3.486042022705078
39.31371307373047 -2.773263931274414 -4.74636697769165 -3.6059508323669434 

38.9177131652832 -2.5964436531066895 -4.580325603485107 -5.4639482498168945 -1.3213801383972168 -1.3213798999786377
38.80685806274414 -2.5869863033294678 -4.578682899475098 -4.139170169830322 -4.215846061706543 -4.215846061706543
38.71257019042969 -2.5407583713531494 -4.484805107116699 -6.590270042419434 -3.5971784591674805 -3.5971784591674805
38.993141174316406 -2.477374315261841 -4.394617080688477 -2.2247121334075928 -3.417051076889038 -3.417051076889038
38.950286865234375 -2.375814437866211 -4.253490447998047 -1.0760568380355835 -5.808567047119141 -5.808567047119141
38.69085693359375 -2.3837103843688965 -4.247262001037598 -5.498861789703369 -4.920860290527344 -4.9208598136901855
38.91142654418945 -2.463881492614746 -4.411641597747803 -2.6419789791107178 -4.495043754577637 -4.4950456619262695
38.96171569824219 -2.5283384323120117 -4.472721099853516 -8.112336158752441 -3.936197519302368 -3.93619704246521
38.7668571472168 -2.5624570846557617 -4.533891201019287 -7.695640563964844 -4.994

...

35.23485565185547 -1.8044185638427734 -3.504706859588623 -7.157718658447266 -9.282793998718262 -9.282793045043945
35.1485710144043 -1.7612264156341553 -3.465131998062134 -3.6115479469299316 -2.2263383865356445 -2.2263379096984863
35.207427978515625 -1.765221118927002 -3.4230828285217285 -6.213537216186523 -10.897159576416016 -10.897157669067383
35.281715393066406 -1.7446969747543335 -3.414104700088501 -3.2303617000579834 -2.4105982780456543 -2.4105987548828125
35.3851432800293 -1.7349815368652344 -3.4057352542877197 -4.8254523277282715 -2.6844990253448486 -2.6844987869262695
35.286285400390625 -1.7218091487884521 -3.3751261234283447 -3.9655559062957764 -6.104237079620361 -6.104238510131836
35.18000030517578 -1.658817172050476 -3.3280208110809326 -4.87504243850708 -2.9663949012756348 -2.9663968086242676
35.1879997253418 -1.6815396547317505 -3.316498041152954 -3.1630442142486572 -3.5597379207611084 -3.5597376823425293
35.3154296875 -1.6837252378463745 -3.310124635696411 -0.76799863576889

35.27428436279297 -1.7683188915252686 -3.4458961486816406 -3.5596725940704346 -1.3666499853134155 -1.3666507005691528
35.27485656738281 -1.7420796155929565 -3.3811018466949463 -3.437352180480957 -4.467566013336182 -4.467565536499023
35.3388557434082 -1.7568048238754272 -3.4495081901550293 -2.1913681030273438 -3.4718313217163086 -3.4718329906463623
35.30400085449219 -1.7584257125854492 -3.4011056423187256 -6.573733806610107 -3.67434024810791 -3.67434024810791
35.29199981689453 -1.7915376424789429 -3.5068142414093018 -0.9414991140365601 -0.5694283246994019 -0.569428563117981
35.19942855834961 -1.7569081783294678 -3.4320785999298096 -3.993380546569824 -0.8232033252716064 -0.823204517364502
35.294857025146484 -1.7578054666519165 -3.4668755531311035 -3.455766201019287 -8.0657320022583 -8.0657320022583
35.2725715637207 -1.77194082736969 -3.4962122440338135 -6.034764766693115 -3.057669162750244 -3.0576682090759277
35.25199890136719 -1.7234578132629395 -3.389279842376709 -4.738239288330078 -1.

35.25371551513672 -1.787983775138855 -3.5027284622192383 -3.8570995330810547 -1.7612671852111816 -1.7612671852111816
35.17828369140625 -1.7595685720443726 -3.4811086654663086 -6.526049613952637 -4.947269439697266 -4.947266578674316
35.22571563720703 -1.8421294689178467 -3.5794360637664795 -5.868534564971924 -7.081118583679199 -7.081120014190674
35.27428436279297 -1.8315229415893555 -3.59818172454834 -2.822723865509033 -2.521022319793701 -2.5210227966308594
35.25199890136719 -1.8520056009292603 -3.6489367485046387 -3.7132139205932617 -3.175130844116211 -3.175131320953369
35.30514144897461 -1.8462438583374023 -3.6280910968780518 -6.0871686935424805 -2.102646827697754 -2.1026463508605957
35.19428634643555 -1.8462767601013184 -3.584573745727539 -1.49418044090271 -1.707162857055664 -1.707162857055664
35.277713775634766 -1.8701088428497314 -3.6325910091400146 -4.767629623413086 -3.95688533782959 -3.9568850994110107
35.290287017822266 -1.802207112312317 -3.522969961166382 -5.44816255569458 -3

35.26856994628906 -1.8688974380493164 -3.6132853031158447 -0.9943615198135376 -4.57744836807251 -4.577450275421143
35.383426666259766 -1.84322988986969 -3.564393997192383 -5.946449279785156 -3.974327564239502 -3.9743285179138184
35.16628646850586 -1.7544963359832764 -3.394836664199829 -5.223928451538086 -0.8827772736549377 -0.882777988910675
35.37885665893555 -1.7516206502914429 -3.4147894382476807 -3.827468156814575 -1.4745442867279053 -1.4745442867279053
35.381141662597656 -1.7144562005996704 -3.344985246658325 -6.192627429962158 -1.843369722366333 -1.8433706760406494
35.31142807006836 -1.685586929321289 -3.2985503673553467 -2.6567516326904297 -1.9025176763534546 -1.9025191068649292
35.402854919433594 -1.662235975265503 -3.2788641452789307 -3.872809410095215 -0.8083803057670593 -0.8083783388137817
35.229713439941406 -1.6778161525726318 -3.280649423599243 -2.623279094696045 -5.670904159545898 -5.670904159545898
35.3725700378418 -1.703342318534851 -3.3508617877960205 -1.489594459533691

KeyboardInterrupt: 

In [32]:
NN.load_state_dict(torch.load('3D_20points_125'))

<All keys matched successfully>

In [33]:
def evaluate(epochs = 30000, npoints = 14, batchsize = 100, nsamples = 8):
    NN.eval()
    with torch.no_grad():
        for i in range(epochs):
            env.reset(npoints, batchsize, nsamples)
            """include the boundary points, kinda makes sense that they should contribute (atm only in the encoder, difficult to see how in the decoder)"""
            memory = NN.encode(env.points) #[npoints, batchsize * nsamples, emsize]
            #### #### #### remember to include tgt.detach() when reinstate with torch.no_grad()
            tgt = NN.start_token.unsqueeze(0).unsqueeze(1).expand(1, batchsize * nsamples, -1).detach() #[1, batchsize * nsamples, emsize]
            #with torch.no_grad(): #to speed up computation, selecting routes is done without gradient
            for j in range(4, npoints):
                #### #### #### remember to include memory.detach() when reinstate with torch.no_grad()
                _, logits = NN.decode_next(memory.detach(), tgt, env.points_mask)
                next_point = env.sampleandgreedy_point(logits)
                """
                for inputing the previous embedding into decoder
                """
                tgt = torch.cat([tgt, memory.gather(0, next_point.unsqueeze(0).unsqueeze(2).expand(1, -1, memory.size(2)))]) #[nsofar, batchsize * nsamples, emsize]
                """
                for inputing the previous decoder output into the decoder (allows for an evolving strategy, but doesn't allow for fast training
                """
                ############


            NN.eval()
            _, logprob = NN.calculate_logprob(memory, env.points_sequence) #[batchsize * nsamples]
            NN.train()
            """
            clip logprob so doesn't reinforce things it already knows
            TBH WANT SOMETHING DIFFERENT ... want to massively increase training if find something unexpected and otherwise not
            """
            
            greedy_baseline = env.cost.view(batchsize, nsamples)[:, -1] #[batchsize], greedy sample
            
            print(greedy_baseline.mean().item(), logprob.view(batchsize, nsamples)[:, -1].mean().item(), logprob.view(batchsize, nsamples)[:, :-1].mean().item(), logprob[batchsize - 1].item(), logprob[0].item(), env.logprob[0].item())
        
evaluate(epochs = 10, npoints = 24, batchsize = 100, nsamples = 2)

123.25 -34.859619140625 -41.079105377197266 -34.52480697631836 -41.39210510253906 -41.39210891723633
122.61000061035156 -34.80966567993164 -40.90222930908203 -34.62474060058594 -40.517799377441406 -40.51779556274414
122.56999969482422 -34.84543991088867 -40.955562591552734 -34.43359375 -40.674259185791016 -40.674259185791016
120.9699935913086 -34.91025161743164 -41.380332946777344 -34.78977584838867 -41.250030517578125 -41.250022888183594
121.55999755859375 -34.84372329711914 -40.860233306884766 -34.94569778442383 -42.449729919433594 -42.449729919433594
121.48999786376953 -34.9001350402832 -41.19115447998047 -35.90367126464844 -41.5755729675293 -41.57557678222656
121.45999908447266 -34.82756423950195 -41.0478515625 -33.66057205200195 -39.6182975769043 -39.6182975769043
122.93000030517578 -34.88264465332031 -41.033119201660156 -34.9913444519043 -41.31243133544922 -41.31242752075195


KeyboardInterrupt: 

537.6380004882812 -138.98419189453125 -147.93531799316406 -146.45074462890625 -149.61941528320312 -149.61941528320312
532.5540161132812 -138.10366821289062 -147.8144989013672 -146.654541015625 -150.35662841796875 -150.3566436767578
530.2640380859375 -136.0601348876953 -147.40367126464844 -146.42555236816406 -147.881103515625 -147.88108825683594
525.9000244140625 -134.02139282226562 -146.74395751953125 -145.3341064453125 -148.57504272460938 -148.5750274658203
524.9900512695312 -132.3279571533203 -146.12123107910156 -145.25173950195312 -148.4559783935547 -148.45594787597656
524.1719970703125 -131.33157348632812 -145.742431640625 -141.67303466796875 -142.32098388671875 -142.32098388671875
521.322021484375 -130.02227783203125 -145.23062133789062 -144.06594848632812 -145.3900146484375 -145.38998413085938
519.8140258789062 -127.49968719482422 -144.056640625 -144.09527587890625 -144.72189331054688 -144.72189331054688
521.4620361328125 -124.63572692871094 -142.46963500976562 -139.2878875732422

514.114013671875 -122.0224380493164 -140.5898895263672 -144.7041778564453 -137.571533203125 -137.57154846191406
514.2300415039062 -122.39403533935547 -140.8399200439453 -143.66539001464844 -143.32257080078125 -143.32257080078125
513.634033203125 -123.73143005371094 -141.79444885253906 -141.58401489257812 -133.7679443359375 -133.76792907714844
517.447998046875 -125.28252410888672 -142.63766479492188 -143.10848999023438 -138.99832153320312 -138.99832153320312
513.8720092773438 -125.713134765625 -142.92547607421875 -147.26687622070312 -140.062744140625 -140.062744140625
515.4739990234375 -125.82928466796875 -142.84996032714844 -147.9103240966797 -138.15269470214844 -138.15269470214844
514.2980346679688 -125.21936798095703 -142.58120727539062 -143.18414306640625 -144.5225372314453 -144.5225372314453
514.5540161132812 -124.11746215820312 -141.89398193359375 -136.3398895263672 -138.9828338623047 -138.9828338623047
509.9980163574219 -122.69064331054688 -140.72781372070312 -141.65386962890625 

517.2360229492188 -121.35770416259766 -139.65931701660156 -140.25885009765625 -140.28668212890625 -140.2866668701172
515.9640502929688 -120.6246337890625 -139.36024475097656 -138.7896270751953 -140.9609375 -140.9609375
516.5420532226562 -121.35859680175781 -139.78598022460938 -133.77322387695312 -144.1346893310547 -144.13470458984375
514.6600341796875 -123.0822525024414 -141.15008544921875 -142.89846801757812 -141.82235717773438 -141.8223419189453
516.8980102539062 -124.23226928710938 -141.9872589111328 -146.69873046875 -144.220458984375 -144.22042846679688
519.35205078125 -124.6902847290039 -142.33163452148438 -143.93702697753906 -140.84738159179688 -140.84738159179688
516.6680297851562 -123.83184814453125 -141.8417510986328 -144.85797119140625 -148.1279296875 -148.12789916992188
515.9860229492188 -122.53752136230469 -141.01405334472656 -151.07037353515625 -131.84259033203125 -131.84255981445312
515.2900390625 -121.79325866699219 -140.52713012695312 -136.75643920898438 -140.2553710937

517.6420288085938 -118.97877502441406 -138.24066162109375 -130.60134887695312 -134.1087646484375 -134.10874938964844
518.738037109375 -124.51573181152344 -141.74757385253906 -142.69053649902344 -136.154541015625 -136.15457153320312
515.8240356445312 -128.2661590576172 -144.0184326171875 -140.3527374267578 -146.35922241210938 -146.3592529296875
517.842041015625 -130.12281799316406 -144.99697875976562 -148.08236694335938 -148.73707580566406 -148.73707580566406
521.8380126953125 -130.74729919433594 -145.2322235107422 -146.12693786621094 -148.87521362304688 -148.87521362304688
519.666015625 -130.55978393554688 -145.09498596191406 -144.96151733398438 -146.13644409179688 -146.13645935058594
517.1280517578125 -129.133544921875 -144.44830322265625 -140.9606475830078 -145.4196319580078 -145.41964721679688
517.5040283203125 -126.37647247314453 -142.95347595214844 -141.06201171875 -146.38568115234375 -146.3856658935547
514.2920532226562 -122.53421020507812 -140.69271850585938 -146.35997009277344 

517.4120483398438 -123.82118225097656 -141.38987731933594 -146.89895629882812 -141.45980834960938 -141.4597625732422
518.822021484375 -123.80950927734375 -141.34495544433594 -137.90994262695312 -148.436279296875 -148.43624877929688
516.248046875 -123.26822662353516 -140.9853515625 -142.35763549804688 -142.51638793945312 -142.51641845703125
518.2140502929688 -122.01557922363281 -140.2266082763672 -139.44943237304688 -134.66729736328125 -134.66726684570312
518.3680419921875 -120.83914184570312 -139.39801025390625 -135.07568359375 -138.61094665527344 -138.6109161376953
517.5 -119.63032531738281 -138.55772399902344 -139.40579223632812 -141.53213500976562 -141.5321044921875
517.9000244140625 -118.58738708496094 -137.7484130859375 -137.62754821777344 -143.09652709960938 -143.09654235839844
516.5680541992188 -118.34347534179688 -137.69622802734375 -141.22186279296875 -129.09507751464844 -129.09506225585938
515.93603515625 -119.19261169433594 -138.05267333984375 -138.06590270996094 -133.354431

...

474.74603271484375 -118.51860809326172 -135.88931274414062 -139.9208984375 -139.90948486328125 -139.9095001220703
475.1040344238281 -118.90074920654297 -135.956298828125 -124.94137573242188 -119.06703186035156 -119.0670394897461
476.988037109375 -119.56273651123047 -136.3390350341797 -115.97672271728516 -144.2639617919922 -144.2639923095703
474.8620300292969 -117.9942398071289 -136.15167236328125 -146.6420135498047 -138.3762664794922 -138.37625122070312
474.87200927734375 -119.52790069580078 -136.44058227539062 -145.66236877441406 -125.95634460449219 -125.95633697509766
475.1960144042969 -118.86402130126953 -136.90997314453125 -147.55718994140625 -129.9114990234375 -129.91148376464844
477.58203125 -118.91619873046875 -136.51210021972656 -135.14212036132812 -146.29425048828125 -146.2942657470703
476.96002197265625 -118.75484466552734 -136.40818786621094 -141.23934936523438 -141.595703125 -141.595703125
477.73602294921875 -119.00421142578125 -136.37574768066406 -123.55919647216797 -132.6

475.8060302734375 -117.3343276977539 -135.7582550048828 -117.61235809326172 -134.14395141601562 -134.14393615722656
476.572021484375 -117.4439468383789 -135.1409912109375 -139.37796020507812 -121.90357208251953 -121.90357971191406
473.8440246582031 -115.84695434570312 -134.5457305908203 -128.27639770507812 -141.3642578125 -141.36427307128906
475.5160217285156 -113.50260162353516 -133.82278442382812 -131.582763671875 -137.40386962890625 -137.40386962890625
475.91802978515625 -113.81367492675781 -133.22561645507812 -130.14431762695312 -143.21649169921875 -143.21649169921875
474.32403564453125 -113.27615356445312 -132.1218719482422 -137.44821166992188 -137.07672119140625 -137.0767364501953
475.4040222167969 -111.12335205078125 -131.02964782714844 -143.1667022705078 -117.9443359375 -117.9443359375
475.99603271484375 -110.72599792480469 -130.18612670898438 -116.58932495117188 -113.81703186035156 -113.81702423095703
473.8260192871094 -108.07534790039062 -129.41461181640625 -112.2367858886718

476.0260314941406 -107.1775894165039 -127.5571517944336 -122.70855712890625 -136.74765014648438 -136.74765014648438
473.3280334472656 -107.52568054199219 -127.57351684570312 -119.95671081542969 -135.73617553710938 -135.7362060546875
472.0400085449219 -107.79163360595703 -128.61788940429688 -139.34007263183594 -124.95552062988281 -124.95553588867188
476.3680114746094 -109.2914810180664 -129.5164031982422 -144.11598205566406 -118.0303955078125 -118.03038024902344
475.1300354003906 -110.56273651123047 -129.9866943359375 -139.6248779296875 -117.41993713378906 -117.4199447631836
477.9420166015625 -111.87290954589844 -131.28060913085938 -138.83499145507812 -124.1456069946289 -124.14562225341797
475.7120361328125 -112.48957061767578 -132.54580688476562 -116.09994506835938 -142.54771423339844 -142.54769897460938
476.5160217285156 -114.36864471435547 -133.35357666015625 -138.0036163330078 -146.16384887695312 -146.163818359375
475.8880310058594 -115.32329559326172 -134.01327514648438 -139.682891

480.030029296875 -125.09378051757812 -140.47547912597656 -147.04254150390625 -130.3365936279297 -130.3365936279297
479.86602783203125 -124.23080444335938 -140.4957733154297 -146.29443359375 -133.8443603515625 -133.8443603515625
480.5680236816406 -124.4832992553711 -140.25852966308594 -132.14427185058594 -142.08010864257812 -142.08010864257812
480.8700256347656 -123.87772369384766 -140.2682342529297 -137.91571044921875 -146.300048828125 -146.300048828125
477.20001220703125 -123.17671203613281 -139.9962615966797 -133.4735107421875 -139.21966552734375 -139.21963500976562
476.27801513671875 -123.42869567871094 -139.58163452148438 -145.26992797851562 -125.89013671875 -125.8901138305664
478.82403564453125 -121.82543182373047 -139.19761657714844 -142.06362915039062 -134.86532592773438 -134.86534118652344
477.384033203125 -121.13880157470703 -138.52719116210938 -144.26211547851562 -145.85040283203125 -145.85044860839844
476.9580078125 -120.56843566894531 -138.27410888671875 -144.74765014648438

475.5180358886719 -111.0398941040039 -130.6012725830078 -137.40911865234375 -103.77496337890625 -103.77497100830078
474.468017578125 -110.32850646972656 -129.97198486328125 -131.92922973632812 -138.48196411132812 -138.48196411132812
474.4220275878906 -109.77772521972656 -129.60523986816406 -141.84153747558594 -119.42591857910156 -119.42591094970703
471.2820129394531 -108.74574279785156 -129.03005981445312 -125.07832336425781 -132.55136108398438 -132.55137634277344
472.1920166015625 -109.15609741210938 -129.16534423828125 -121.9928970336914 -115.1138916015625 -115.11389923095703
473.7640075683594 -109.5073013305664 -129.2136993408203 -123.59307861328125 -114.3445053100586 -114.34451293945312
474.39801025390625 -108.98274230957031 -129.41400146484375 -135.31671142578125 -141.7877197265625 -141.7877197265625
474.8580322265625 -110.29557037353516 -130.4092254638672 -121.16719818115234 -127.50227355957031 -127.50228118896484
473.96002197265625 -109.23457336425781 -130.39535522460938 -122.50

In [2]:
"""
REDO THIS WITHOUT KEEPING TRACK OF EDGES

Idea: among removed triangles, pair up faces that both apear, left with faces that don't - the boundary, from which we construct new triangles

have two lists, faces left to check, and faces to check against (these will be all 3 anticlockwise versions of each face)
keep track of the batch you came from, and the index against which you are currently checking
increase index by one each time until either: find a match, or: no longer checking against same batch
at which point we remove FROM THE FIRST LIST
repeat until all removed
when find a match, mark it in second list
removed all marked faces
somehow find number remaining in each batch, and make sure to copy that many 'new points' into a long list
construct new triangles from the above information
"""



import math
import random
import numpy as np
import torch


device = "cuda:0"
floattype = torch.float


class environment:    
    def reset(self, npoints, batchsize, nsamples=1, corner_points = None, initial_triangulation = None):
        """
        corner_points, etc., shoudn't include a batch dimension
        """
        if corner_points == None:
            ncornerpoints = 4
        else:
            ncornerpoints = corner_points.size(0)
        if npoints <= ncornerpoints:
            print("Error: not enough points for valid problem instance")
            return
        self.batchsize = (
            batchsize * nsamples
        )  # so that I don't have to rewrite all this code, we store these two dimensions together
        self.nsamples = nsamples
        self.npoints = npoints
        self.points = (
            torch.rand([batchsize, npoints - ncornerpoints, 3], dtype = floattype, device=device)
            .unsqueeze(1)
            .expand(-1, nsamples, -1, -1)
            .reshape(self.batchsize, npoints - ncornerpoints, 3)
        )
        if corner_points == None:
            self.corner_points = torch.tensor(
                [[0, 0, 0], [3, 0, 0], [0, 3, 0], [0, 0, 3]], dtype = floattype, device=device
            )
        else:
            self.corner_points = corner_points
        self.points = torch.cat(
            [
                self.corner_points.unsqueeze(0).expand(self.batchsize, -1, -1),
                self.points,
            ],
            dim=-2,
        )  # [batchsize * nsamples, npoints, 3]
        self.points_mask = torch.cat(
            [
                torch.ones([self.batchsize, ncornerpoints], dtype=torch.bool, device=device),
                torch.zeros(
                    [self.batchsize, npoints - ncornerpoints], dtype=torch.bool, device=device
                ),
            ],
            dim=1,
        )
        self.points_sequence = torch.empty(
            [self.batchsize, 0], dtype=torch.long, device=device
        )

        """
        points are now triples
        triangles are now quadruples
        edges are now still just indices, but there are four of them per 'triangle', and they correspond to triples of points, not pairs
        we use  0,2,1  0,3,2  0,1,3  1,2,3  as the order of the four 'edges'/faces
        opposite face is always ordered such that the last two indices are swapped
        faces are always read ANTICLOCKWISE
        
        first three points of tetrahedron MUST be read clockwise (from the outside) to get correct sign on incircle test
        
        new point will be inserted in zeroth position, so if corresponding face of REMOVED tetrahedron is [x,y,z] (being read anticlockwise from outside in) new tetrahedron is [p, x, y, z]
        """
        
        """
        number of tetrahedra is not the same for each batch (in 3D), so store as a big list, and remember batch index that it comes from
        """
        if corner_points == None:
            initial_triangulation = torch.tensor([[0, 1, 2, 3]], dtype=torch.long, device=device)
        
        self.partial_delaunay_triangles = initial_triangulation.unsqueeze(0).expand(self.batchsize, -1, -1).reshape(-1, 4)
        self.batch_index = torch.arange(self.batchsize, dtype = torch.long, device = device).unsqueeze(1).expand(-1, initial_triangulation.size(0)).reshape(-1)
        
        self.batch_triangles = self.partial_delaunay_triangles.size(0) #[0]
        self.ntriangles = torch.full([self.batchsize], initial_triangulation.size(0), dtype = torch.long, device = device) #[self.batchsize]
        
        self.cost = torch.zeros([self.batchsize], dtype = floattype, device=device)

        self.logprob = torch.zeros([self.batchsize], dtype = floattype, device=device, requires_grad=True)

    def update(self, point_index):  # point_index is [batchsize]
        
        assert point_index.size(0) == self.batchsize
        assert str(point_index.device) == device
        assert self.points_mask.gather(1, point_index.unsqueeze(1)).sum() == 0
        
        triangles_coordinates = self.points[self.batch_index.unsqueeze(1), self.partial_delaunay_triangles] # [batch_triangles, 4, 3]
        
        newpoint = self.points[self.batch_index, point_index[self.batch_index]] # [batch_triangles, 3]

        incircle_matrix = torch.cat(
            [
                newpoint.unsqueeze(1),
                triangles_coordinates,
            ],
            dim=-2,
        )  # [batch_triangles, 5, 3]
        incircle_matrix = torch.cat(
            [
                (incircle_matrix * incircle_matrix).sum(-1, keepdim=True),
                incircle_matrix,
                torch.ones([self.batch_triangles, 5, 1], dtype = floattype, device=device),
            ],
            dim=-1,
        )  # [batch_triangles, 5, 5]
        assert incircle_matrix.dtype == floattype
        assert str(incircle_matrix.device) == device
        
        incircle_test = (
            incircle_matrix.det() > 0
        )  # [batch_triangles], is True if inside incircle
        
        conflicts = incircle_test.sum()
        
        conflicting_triangles = self.partial_delaunay_triangles[incircle_test] # [conflicts, 4]
        
        conflicting_edges_index0 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([0, 0, 0, 1])
        conflicting_edges_index0 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges_index1 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([2, 3, 1, 2])
        conflicting_edges_index1 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges_index2 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([1, 2, 3, 3])
        conflicting_edges_index2 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges = torch.cat([conflicting_edges_index0.view(-1).unsqueeze(-1), conflicting_edges_index1.view(-1).unsqueeze(-1), conflicting_edges_index2.view(-1).unsqueeze(-1)], dim = -1).reshape(-1, 3) # [conflicts * 4, 3]
        
        edge_batch_index = self.batch_index[incircle_test].unsqueeze(1).expand(-1, 4).reshape(-1) # [conflicts * 4]
        
        indices = torch.LongTensor([0, 2, 1])
        comparison_edges = conflicting_edges[:, indices] # [conflicts * 4, 3]        
        
        unravel_nomatch_mask = torch.ones([conflicts * 4], dtype = torch.bool, device = device) # [conflicts * 4]
        i = 1
        while True:
            
            todo_mask = unravel_nomatch_mask[:-i].logical_and(edge_batch_index[:-i] == edge_batch_index[i:])
            if i % 4 == 0:
                if todo_mask.sum() == 0:
                    break
            
            match_mask = todo_mask.clone()
            match_mask[todo_mask] = (conflicting_edges[:-i][todo_mask] != comparison_edges[i:][todo_mask]).sum(-1).logical_not()
            
            unravel_nomatch_mask[:-i][match_mask] = False
            unravel_nomatch_mask[i:][match_mask] = False
            
            i += 1
        
        batch_newtriangles = unravel_nomatch_mask.sum()
        
        nomatch_edges = conflicting_edges[unravel_nomatch_mask] # [batch_newtriangles, 3], already in correct order to insert into 1,2,3 (since already anticlockwise from outside in)
        assert list(nomatch_edges.size()) == [batch_newtriangles, 3]
        nomatch_batch_index = edge_batch_index[unravel_nomatch_mask] # [batch_newtriangles]
        
        nomatch_newpoint = point_index[nomatch_batch_index] # [batch_newtriangles]
        
        newtriangles = torch.cat([nomatch_newpoint.unsqueeze(1), nomatch_edges], dim = -1) # [batch_newtriangles, 4]
        
        
        nremoved_triangles = torch.zeros([self.batchsize], dtype = torch.long, device = device)
        nnew_triangles = torch.zeros([self.batchsize], dtype = torch.long, device = device)
        
        indices = self.batch_index[incircle_test]
        nremoved_triangles.put_(indices, torch.ones_like(indices, dtype = torch.long), accumulate = True) # [batchsize]
        
        indices = edge_batch_index[unravel_nomatch_mask]
        nnew_triangles.put_(indices, torch.ones_like(indices, dtype = torch.long), accumulate = True) # [batchsize]
        
        assert (nnew_triangles <= 2 * nremoved_triangles + 2).logical_not().sum().logical_not()
        
        """
        NOTE:
        I THINK it's possible for nnew_triangles to be less than nremoved_triangles (or my code is just buggy...)
        """
        
        assert nnew_triangles.sum() == batch_newtriangles
        assert nremoved_triangles.sum() == incircle_test.sum()
        
        nadditional_triangles = nnew_triangles - nremoved_triangles # [batchsize]
        ntriangles = self.ntriangles + nadditional_triangles # [batchsize]
        
        partial_delaunay_triangles = torch.empty([ntriangles.sum(), 4], dtype = torch.long, device = device)
        batch_index = torch.empty([ntriangles.sum()], dtype = torch.long, device = device)
        
        cumulative_triangles = torch.cat([torch.zeros([1], dtype = torch.long, device = device), nnew_triangles.cumsum(0)[:-1]]) # [batchsize], cumulative sum starts at zero
        
        """
        since may actually have LESS triangles than previous round, we insert all that survive into the first slots (in that batch)
        """
        good_triangle_indices = torch.arange(incircle_test.logical_not().sum(), dtype = torch.long, device = device)
        good_triangle_indices += cumulative_triangles[self.batch_index[incircle_test.logical_not()]]
        bad_triangle_indices_mask = torch.ones([ntriangles.sum(0)], dtype = torch.bool, device = device)
        bad_triangle_indices_mask.scatter_(0, good_triangle_indices, False)
        
        assert good_triangle_indices.size(0) == incircle_test.logical_not().sum()
        assert bad_triangle_indices_mask.sum() == batch_newtriangles
        
        partial_delaunay_triangles[good_triangle_indices] = self.partial_delaunay_triangles[~incircle_test]
        batch_index[good_triangle_indices] = self.batch_index[~incircle_test]
        
        partial_delaunay_triangles[bad_triangle_indices_mask] = newtriangles
        batch_index[bad_triangle_indices_mask] = nomatch_batch_index
        
        self.partial_delaunay_triangles = partial_delaunay_triangles
        self.batch_index = batch_index
        
        self.ntriangles = ntriangles
        self.batch_triangles = self.partial_delaunay_triangles.size(0)
        
        self.points_mask.scatter_(
            1, point_index.unsqueeze(1).expand(-1, self.npoints), True
        )
        self.points_sequence = torch.cat(
            [self.points_sequence, point_index.unsqueeze(1)], dim=1
        )
        
        self.cost += nremoved_triangles
        return
    
    def allindices(self): #generate all orders of point insertion
        npoints = self.npoints - 4
        allroutes = torch.empty([1, 0], dtype = torch.long, device = device)
        for i in range(npoints):
            nroutes = allroutes.size(0)
            remaining_mask = torch.ones([nroutes], dtype = torch.bool, device = device).unsqueeze(1).expand(-1, npoints).clone().scatter_(-1, allroutes, False)
            remaining_indices = remaining_mask.nonzero(as_tuple = True)[1]
            allroutes = allroutes.unsqueeze(1).expand(-1, remaining_mask[0, :].sum(), -1)
            allroutes = torch.cat([allroutes, remaining_indices.view(nroutes, -1).unsqueeze(2)], dim = -1).view(-1, allroutes.size(-1) + 1)
        return allroutes #[npoints!, npoints]


# turn integer x,y coords (in nxn grid) into position d (0 to n^2-1) along the Hilbert curve.
def xy2d(n, x, y):
    [x, y] = [math.floor(x), math.floor(y)]
    [rx, ry, s, d] = [0, 0, 0, 0]
    s = n / 2
    s = math.floor(s)
    while s > 0:
        rx = (x & s) > 0  # bitwise and, and then boolean is it greater than 0?
        ry = (y & s) > 0
        d += s * s * ((3 * rx) ^ ry)
        [x, y] = rot(n, x, y, rx, ry)
        s = s / 2
        s = math.floor(s)
    return d


def rot(n, x, y, rx, ry):
    if ry == 0:
        if rx == 1:
            x = n - 1 - x
            y = n - 1 - y
        # Swap x and y
        t = x
        x = y
        y = t
    return x, y


def order(
    n, points
):  # turns tensor of points into integer distances along hilbert curve of itteration n
    grid = n * points.to("cpu")
    x = torch.empty([grid.size(0)])
    for i in range(points.size(0)):
        x[i] = xy2d(n, grid[i, 0], grid[i, 1])
    return x

"""
CURRENTLY ONLY 2D VERSION
"""
def hilbert_insertion(npoints=103, batchsize=200):
    env.reset(npoints, batchsize)
    points = env.points[:, 3:]  # [batchsize, npoints - 3]
    insertion_order = torch.full([batchsize, npoints], float("inf"), device=device)
    for i in range(batchsize):
        insertion_order[i, 3:] = order(
            2 ** 6, points[i]
        )  # number of possible positions is n ** 2
    for i in range(npoints - 3):
        next_index = insertion_order.min(-1)[1]
        env.update(next_index)
        insertion_order.scatter_(1, next_index.unsqueeze(1), float("inf"))
    print(env.cost.mean().item(), env.cost.var().sqrt().item())
    return


def random_insertion(npoints=104, batchsize=200):
    env.reset(npoints, batchsize)
    for i in range(npoints - 4):
        env.update(torch.full([batchsize], i + 4, dtype=torch.long, device=device))
    print(env.cost.mean().item(), env.cost.var().sqrt().item())
    return

    """
    UNDER CONSTRUCTION
    """
def kdtree_insertion(npoints=103, batchsize=200):
    env.reset(npoints, batchsize)
    points = env.points[:, 3:]  # [batchsize, npoints - 3]
    
env = environment()

In [2]:
env = environment()
npoints = 9
batchsize = 1
        
env.reset(npoints + 4, batchsize, math.factorial(npoints))
allroutes = env.allindices() + 4
allroutes = allroutes.unsqueeze(0).expand(batchsize, -1, -1).reshape(-1, npoints)
for j in range(10):
    for i in range(npoints):
        env.update(allroutes[:, i])
    print(env.cost.view(batchsize, -1).min(-1)[0].mean().item(), env.cost.mean().item())
    env.reset(npoints + 4, batchsize, math.factorial(npoints))

25.0 43.19682312011719
27.0 42.42301559448242
26.0 43.520633697509766
26.0 41.43492126464844
30.0 40.770633697509766
32.0 42.38730239868164
26.0 39.82460403442383
31.0 41.69682312011719
28.0 40.746826171875
28.0 42.14603042602539


In [2]:
env = environment()
npoints = 8
batchsize = 10
        
env.reset(npoints + 4, batchsize, math.factorial(npoints))
allroutes = env.allindices() + 4
allroutes = allroutes.unsqueeze(0).expand(batchsize, -1, -1).reshape(-1, npoints)
for j in range(10):
    for i in range(npoints):
        env.update(allroutes[:, i])
    print(env.cost.view(batchsize, -1).min(-1)[0].mean().item(), env.cost.mean().item())
    env.reset(npoints + 4, batchsize, math.factorial(npoints))

23.5 34.148094177246094
23.200000762939453 34.55595016479492
23.0 34.28666687011719
22.700000762939453 33.59000015258789
21.80000114440918 34.04595184326172
23.0 33.55976104736328
23.600000381469727 34.18499755859375
22.899999618530273 33.813331604003906
22.200000762939453 34.28190231323242
22.100000381469727 34.10976028442383


In [ ]:
env = environment()
npoints = 10
batchsize = 1
nsamples = 100000
        
env.reset(npoints + 4, nsamples)
allroutes = env.allindices() + 4
allroutes = allroutes.unsqueeze(0).expand(batchsize, -1, -1).reshape(-1, npoints)[:nsamples, :]
print(allroutes.size())
for j in range(1000):
    for i in range(npoints):
        env.update(allroutes[:, i])
    print(env.cost.view(batchsize, -1).min(-1)[0].mean().item(), env.cost.mean().item())
    env.reset(npoints + 4, batchsize, nsamples)

torch.Size([100000, 10])
29.0 49.65717697143555
36.0 48.87055969238281
38.0 51.05009841918945
36.0 51.02914047241211
31.0 45.161277770996094
41.0 52.800697326660156
38.0 49.204898834228516
39.0 50.78003692626953
38.0 49.051597595214844
33.0 47.717559814453125
32.0 46.3649787902832
35.0 47.345619201660156
35.0 45.504600524902344
35.0 49.83915710449219
33.0 44.56079864501953
37.0 51.07265853881836
37.0 52.16703796386719
38.0 49.48624038696289
36.0 51.30870056152344
35.0 46.69947814941406
39.0 53.22419738769531
33.0 48.71516036987305
36.0 52.49327850341797
38.0 51.98114013671875
44.0 52.908878326416016
34.0 49.60095977783203
35.0 50.536598205566406
35.0 51.26199722290039
38.0 49.712860107421875
36.0 53.11921691894531
36.0 49.411720275878906
33.0 49.54199981689453
35.0 47.198917388916016
39.0 49.790977478027344
36.0 54.170318603515625
37.0 49.390159606933594
38.0 49.5429573059082
37.0 48.32611846923828
39.0 51.07919692993164
35.0 49.789737701416016
35.0 51.28583908081055
35.0 47.7252006530

In [3]:
for i in range(10):
    random_insertion(1004, 100)

RuntimeError: cuda runtime error (1) : invalid argument at /opt/conda/conda-bld/pytorch_1614378062065/work/aten/src/THC/THCCachingHostAllocator.cpp:278

In [1]:
import math
import random
import numpy as np
import torch


"""
Install pytorch first, see https://pytorch.org/get-started/locally/ for instructions
"""


"""
If a GPU is available set device to "cuda" below, and this will speed up the code significantly (be careful not to overflow the GPU memory)
To find out if a GPU is available you can use the command
    torch.cuda.is_available()
"""
device = "cuda:0"


"""
To just compute the cost of the Hilbert Curve alogrithm use the function
    hilbert_insertion(npoints, batchsize)
npoints is the total number of points (including the 3 original corner points!!!!)
batchsize is the number of problem instances we triangulate (say 200, then we would have 200 sets of npoints)

This function will just print the average number of 'additional' triangles deleted, and the standard deviation of the number of 'additional' triangles deleted
"""


"""
To just compute the cost of the random insertion alogrithm use the function
    random_insertion(npoints, batchsize)
npoints is the total number of points (including the 3 original corner points!!!!)
batchsize is the number of problem instances we triangulate (say 200, then we would have 200 sets of npoints)

this function will just print the average number of 'additional' triangles deleted, and the standard deviation of the number of 'additional' triangles deleted
"""


"""
This class (below) is the actual delaunay algorithm

Initiate by
    env = environment()
To generate new random points use
    env.reset(npoints, batchsize)
(don't worry about what nsamples does in the actual code below, it's used for training NNs using reinforcement learning)

npoints is the total number of points to be generated, the first 3 are the 3 corners of the big triangle, all the rest are randomly chosen in the unit square
batchsize is how many problems you want to simultaneously create (lets say 200), then you would have 200 sets of npoints, currently all 200 are untriangulated

The current point locations can be found by
    env.points
The current (partial) delaunay triangulations can be found by
    env.triangles
These are stored as triples of indices, so the triangle [0, 2426, 564] refers to the 0th point, 2426th point, and 564th point stored in env.points
Note: the big outside triangle has two copies, a clockwise ([2, 1, 0]) and anticlockwise ([0, 1, 2]) one, the clockwise one will always stay and never get removed - it's used for some trickery for when the delaunay cavity includes one of the external edges, don't worry about it

To add a point to the current (partial) delaunay triangulation use
    env.update(points)
where points is a torch.tensor (see the pytorch website for how to generate and use tensors) of size [batchsize], containing the index of the points you want to add FOR EACH PROBLEM INSTANCE (remember there might be batchsize = 200 problem instances, so you would have to specify 200 points to add, one for each problem instance)
This will update the current (partial) delaunay triangulation, env.triangles

To see the current cost (the 'additional' triangles that had to be removed) use
    env.cost
This will output a torch.tensor of size [batchsize], i.e. a 'list' of costs, corresponding to each of the (say 200) problem instances
"""


class environment:
    def reset(self, npoints, batchsize, nsamples=1):
        if npoints <= 3:
            print("Error: not enough points for valid problem instance")
            return
        self.batchsize = (
            batchsize * nsamples
        )  # so that I don't have to rewrite all this code, we store these two dimensions together
        self.nsamples = nsamples
        self.npoints = npoints
        self.points = (
            torch.rand([batchsize, npoints - 3, 2], device=device)
            .unsqueeze(1)
            .expand(-1, nsamples, -1, -1)
            .reshape(self.batchsize, npoints - 3, 2)
        )
        self.corner_points = torch.tensor(
            [[0, 0], [2, 0], [0, 2]], dtype=torch.float, device=device
        )
        self.points = torch.cat(
            [
                self.corner_points.unsqueeze(0).expand(self.batchsize, -1, -1),
                self.points,
            ],
            dim=-2,
        )  # [batchsize * nsamples, npoints, 2]
        self.points_mask = torch.cat(
            [
                torch.ones([self.batchsize, 3], dtype=torch.bool, device=device),
                torch.zeros(
                    [self.batchsize, npoints - 3], dtype=torch.bool, device=device
                ),
            ],
            dim=1,
        )
        self.points_sequence = torch.empty(
            [self.batchsize, 0], dtype=torch.long, device=device
        )

        """use a trick, for the purpose of an 'external' triangle that is always left untouched, which means we don't have to deal with boundary edges as being different. external triangle is [0, 1, 2] traversed clockwise..."""
        self.partial_delaunay_triangles = (
            torch.tensor([[0, 2, 1], [0, 1, 2]], dtype=torch.int64, device=device)
            .unsqueeze(0)
            .expand(self.batchsize, -1, -1)
            .contiguous()
        )  # [batchsize, ntriangles, 3] contains index of points, always anticlockwise
        self.partial_delaunay_edges = (
            torch.tensor([5, 4, 3, 2, 1, 0], dtype=torch.int64, device=device)
            .unsqueeze(0)
            .expand(self.batchsize, -1)
            .contiguous()
        )  # [batchsize, ntriangles * 3] contains location of corresponding edge (edges go in order 01, 12, 20). Edges will always flip since triangles are stored anticlockwise.

        self.ntriangles = 2  # can store as scalar, since will always be the same
        self.cost = torch.zeros([self.batchsize], device=device)

        self.logprob = torch.zeros([self.batchsize], device=device, requires_grad=True)

    def update(self, point_index):  # point_index is [batchsize]
        if point_index.size(0) != self.batchsize:
            print(
                "Error: point_index.size() doesn't match expected size, should be [batchsize]"
            )
            return
        if self.points_mask.gather(1, point_index.unsqueeze(1)).sum():
            print("Error: some points already added")
            return
        triangles_coordinates = self.points.gather(
            1,
            self.partial_delaunay_triangles.view(self.batchsize, self.ntriangles * 3)
            .unsqueeze(2)
            .expand(-1, -1, 2),
        ).view(
            self.batchsize, self.ntriangles, 3, 2
        )  # [batchsize, ntriangles, 3, 2]
        newpoint = self.points.gather(
            1, point_index.unsqueeze(1).unsqueeze(2).expand(self.batchsize, 1, 2)
        ).squeeze(
            1
        )  # [batchsize, 2]

        incircle_matrix = torch.cat(
            [
                triangles_coordinates,
                newpoint.unsqueeze(1).unsqueeze(2).expand(-1, self.ntriangles, 1, -1),
            ],
            dim=-2,
        )  # [batchsize, ntriangles, 4, 2]
        incircle_matrix = torch.cat(
            [
                incircle_matrix,
                (incircle_matrix * incircle_matrix).sum(-1, keepdim=True),
                torch.ones([self.batchsize, self.ntriangles, 4, 1], device=device),
            ],
            dim=-1,
        )  # [batchsize, ntriangles, 4, 4]
        incircle_test = (
            incircle_matrix.det() > 0
        )  # [batchsize, ntriangles], is True if inside incircle
        removed_edge_mask = (
            incircle_test.unsqueeze(2).expand(-1, -1, 3).reshape(-1)
        )  # [batchsize * ntriangles * 3]

        edges = (
            self.partial_delaunay_edges
            + self.ntriangles
            * 3
            * torch.arange(self.batchsize, device=device).unsqueeze(1)
        ).view(
            -1
        )  # [batchsize * ntriangles * 3]
        neighbouring_edge = edges.masked_select(removed_edge_mask)
        neighbouring_edge_mask = torch.zeros(
            [self.batchsize * self.ntriangles * 3], device=device, dtype=torch.bool
        )
        neighbouring_edge_mask[neighbouring_edge] = True
        neighbouring_edge_mask = (
            neighbouring_edge_mask * removed_edge_mask.logical_not()
        )  # [batchsize * ntriangles * 3]

        n_new_triangles = neighbouring_edge_mask.view(self.batchsize, -1).sum(
            -1
        )  # [batchsize]

        new_point = (
            point_index.unsqueeze(1)
            .expand(-1, self.ntriangles * 3)
            .masked_select(neighbouring_edge_mask.view(self.batchsize, -1))
        )

        second_point_mask = neighbouring_edge_mask.view(
            self.batchsize, -1, 3
        )  # [batchsize, ntriangles 3]
        (
            first_point_indices0,
            first_point_indices1,
            first_point_indices2,
        ) = second_point_mask.nonzero(as_tuple=True)
        first_point_indices2 = (first_point_indices2 != 2) * (first_point_indices2 + 1)

        first_point = self.partial_delaunay_triangles[
            first_point_indices0, first_point_indices1, first_point_indices2
        ]  # [?]
        second_point = self.partial_delaunay_triangles.masked_select(
            second_point_mask
        )  # [?]

        new_triangles_mask = torch.cat(
            [
                incircle_test,
                torch.ones([self.batchsize, 2], dtype=torch.bool, device=device),
            ],
            dim=1,
        )  # [batchsize, ntriangles + 2]

        new_neighbouring_edges = (
            3 * new_triangles_mask.nonzero(as_tuple=True)[1]
        )  # [?], 3* since is the 01 edge of new triangles (see later)
        self.partial_delaunay_edges.masked_scatter_(
            neighbouring_edge_mask.view(self.batchsize, -1), new_neighbouring_edges
        )  # still [batchsize, ntriangles * 3] for now

        self.partial_delaunay_triangles = torch.cat(
            [
                self.partial_delaunay_triangles,
                torch.empty([self.batchsize, 2, 3], dtype=torch.long, device=device),
            ],
            dim=1,
        )
        self.partial_delaunay_edges = torch.cat(
            [
                self.partial_delaunay_edges,
                torch.empty([self.batchsize, 6], dtype=torch.long, device=device),
            ],
            dim=1,
        )
        new_triangles = torch.stack(
            [first_point, second_point, new_point], dim=1
        )  # [?, 3], edge here is flipped compared to edge in neighbouring triangle (so first_point is the second point in neighbouring edge)
        self.partial_delaunay_triangles.masked_scatter_(
            new_triangles_mask.unsqueeze(2).expand(-1, -1, 3), new_triangles
        )  # [batchsize, ntriangles + 2, 3]

        new_edge01 = neighbouring_edge_mask.view(self.batchsize, -1).nonzero(
            as_tuple=True
        )[
            1
        ]  # [?]

        """we are currently storing which triangles have to be inserted, via the edges along the perimeter of the delaunay cavity, we need to compute which edge is to the 'left'/'right' of each edge"""
        """don't have the memory to do a batchsize * n * n boolean search, don't have the speed to do a batchsize^2 search (as would be the case for sparse matrix or similar)"""
        """best alternative: rotate the edge around right point, repeat until hit edge in mask (will never go to an edge of a removed triangle before we hit edge in mask) should basically be order 1!"""

        neighbouring_edge_index = neighbouring_edge_mask.nonzero(as_tuple=True)[
            0
        ]  # [?]
        next_neighbouring_edge_index = torch.empty_like(neighbouring_edge_index)  # [?]

        rotating_flipped_neighbouring_edge_index = neighbouring_edge_mask.nonzero(
            as_tuple=True
        )[
            0
        ]  # [?], initialise
        todo_mask = torch.ones_like(
            next_neighbouring_edge_index, dtype=torch.bool
        )  # [?]
        while todo_mask.sum():
            rotating_neighbouring_edge_index = (
                rotating_flipped_neighbouring_edge_index
                + 1
                - 3 * (rotating_flipped_neighbouring_edge_index % 3 == 2)
            )  # [todo_mask.sum()], gets smaller until nothing left EFFICIENCY (this may be seriously stupid, as it requires making a bunch of copies when I could be doing stuff inplace)

            update_mask = neighbouring_edge_mask[
                rotating_neighbouring_edge_index
            ]  # [todo_mask.sum()]
            update_mask_unravel = torch.zeros_like(todo_mask).masked_scatter(
                todo_mask, update_mask
            )  # [?]

            next_neighbouring_edge_index.masked_scatter_(
                update_mask_unravel,
                rotating_neighbouring_edge_index.masked_select(update_mask),
            )  # [?]

            todo_mask.masked_fill_(update_mask_unravel, False)  # [?]
            rotating_flipped_neighbouring_edge_index = edges[
                rotating_neighbouring_edge_index.masked_select(
                    update_mask.logical_not()
                )
            ]  # [todo_mask.sum()]
        triangle_index = new_triangles_mask.view(-1).nonzero(as_tuple=True)[
            0
        ]  # [?], index goes up to batchsize * (ntriangles + 2), this is needed for when we invert the permutation by scattering (won't scatter same number of triangles per batch)

        next_triangle_index = torch.empty_like(edges).masked_scatter_(
            neighbouring_edge_mask, triangle_index
        )[
            next_neighbouring_edge_index
        ]  # [?], index goes up to batchsize * (ntriangles + 2)
        next_edge = 3 * next_triangle_index + 1  # [?]

        invert_permutation = torch.empty_like(
            new_triangles_mask.view(-1), dtype=torch.long
        )  # [batchsize * (ntriangles + 2)]
        invert_permutation[
            next_triangle_index
        ] = triangle_index  # [batchsize * (ntriangles + 2)]
        previous_triangle_index = invert_permutation.masked_select(
            new_triangles_mask.view(-1)
        )  # [?]
        previous_edge = 3 * previous_triangle_index + 2  # [?]

        """in the above we rotated around 'first_point' in our new triangles"""
        new_edge20 = next_edge % ((self.ntriangles + 2) * 3)  # [?]
        new_edge12 = previous_edge % ((self.ntriangles + 2) * 3)  # [?]

        new_edges = torch.stack([new_edge01, new_edge12, new_edge20], dim=1)  # [?, 3]
        self.partial_delaunay_edges.masked_scatter_(
            new_triangles_mask.unsqueeze(2)
            .expand(-1, -1, 3)
            .reshape(self.batchsize, -1),
            new_edges,
        )  # [batchsize, (ntriangles + 2) * 3]

        self.ntriangles += 2
        """currently only count the extra triangles you replace (not the one you have to remove because you're located there, and not the ones you make because you have to create two more"""
        self.cost += n_new_triangles - 3
        self.points_mask.scatter_(
            1, point_index.unsqueeze(1).expand(-1, self.npoints), True
        )
        self.points_sequence = torch.cat(
            [self.points_sequence, point_index.unsqueeze(1)], dim=1
        )


# turn integer x,y coords (in nxn grid) into position d (0 to n^2-1) along the Hilbert curve.
def xy2d(n, x, y):
    [x, y] = [math.floor(x), math.floor(y)]
    [rx, ry, s, d] = [0, 0, 0, 0]
    s = n / 2
    s = math.floor(s)
    while s > 0:
        rx = (x & s) > 0  # bitwise and, and then boolean is it greater than 0?
        ry = (y & s) > 0
        d += s * s * ((3 * rx) ^ ry)
        [x, y] = rot(n, x, y, rx, ry)
        s = s / 2
        s = math.floor(s)
    return d


def rot(n, x, y, rx, ry):
    if ry == 0:
        if rx == 1:
            x = n - 1 - x
            y = n - 1 - y
        # Swap x and y
        t = x
        x = y
        y = t
    return x, y


def order(
    n, points
):  # turns tensor of points into integer distances along hilbert curve of itteration n
    grid = n * points.to("cpu")
    x = torch.empty([grid.size(0)])
    for i in range(points.size(0)):
        x[i] = xy2d(n, grid[i, 0], grid[i, 1])
    return x


def hilbert_insertion(npoints=103, batchsize=200):
    env.reset(npoints, batchsize, nsamples=1)
    points = env.points[:, 3:]  # [batchsize, npoints - 3]
    insertion_order = torch.full([batchsize, npoints], float("inf"), device=device)
    for i in range(batchsize):
        insertion_order[i, 3:] = order(
            2 ** 6, points[i]
        )  # number of possible positions is n ** 2
    for i in range(npoints - 3):
        next_index = insertion_order.min(-1)[1]
        env.update(next_index)
        insertion_order.scatter_(1, next_index.unsqueeze(1), float("inf"))
    print(env.cost.mean().item(), env.cost.var().sqrt().item())


def random_insertion(npoints=103, batchsize=200):
    env.reset(npoints, batchsize, nsamples=1)
    for i in range(npoints - 3):
        env.update(torch.full([batchsize], i + 3, dtype=torch.long, device=device))
    print(env.cost.mean().item(), env.cost.var().sqrt().item())

env = environment()

In [ ]:
for i in range(10):
    random_insertion(1003, 100)

2890.909912109375 42.50613021850586
2891.059814453125 42.404930114746094
2890.579833984375 42.23149490356445
